# Bot 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
!pip install python-telegram-bot

     |████████████████████████████████| 327kB 2.8MB/s 
     |████████████████████████████████| 2.3MB 35.6MB/s 
     |████████████████████████████████| 102kB 26.2MB/s 


In [2]:
!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git

Cloning into 'Real-Time-Voice-Cloning'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2325 (delta 0), reused 0 (delta 0), pack-reused 2322
Receiving objects: 100% (2325/2325), 360.24 MiB | 13.05 MiB/s, done.
Resolving deltas: 100% (1274/1274), done.


In [1]:
cd Real-Time-Voice-Cloning/

/content/Real-Time-Voice-Cloning


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 377.0MB 66kB/s 
     |████████████████████████████████| 1.4MB 30.9MB/s 
     |████████████████████████████████| 71kB 22.6MB/s 
     |████████████████████████████████| 245kB 40.4MB/s 
     |████████████████████████████████| 62.1MB 50.4MB/s 
     |████████████████████████████████| 204kB 38.9MB/s 
     |████████████████████████████████| 71kB 24.7MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.8-cp36-none-any.whl size=1350602 sha256=46536a20a4bb364b286e41bff61a25a41854f76d3c4093fec18441c88f940522
  Stored in directory: /root/.cache/pip/wheels/ee/87/ce/a5023722374ca73b57fc8d4284ba6f973c01219b3c385a07e0
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp36-cp36m-linux_x86_64.whl size=71348 sha256=54f1186210a36636c10edb51ba58ec165f775d950408b687bd2ba412ebebbf48
  Stored in directory: /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.wh

In [5]:
!gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc

Downloading...
From: https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
To: /content/Real-Time-Voice-Cloning/pretrained.zip
384MB [00:05, 64.5MB/s]


In [6]:
!unzip pretrained.zip

Archive:  pretrained.zip
   creating: encoder/saved_models/
  inflating: encoder/saved_models/pretrained.pt  
   creating: synthesizer/saved_models/
   creating: synthesizer/saved_models/logs-pretrained/
   creating: synthesizer/saved_models/logs-pretrained/taco_pretrained/
 extracting: synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.meta  
   creating: vocoder/saved_models/
   creating: vocoder/saved_models/pretrained/
  inflating: vocoder/saved_models/pretrained/pretrained.pt  


In [7]:
!apt-get install libportaudio2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 1s (46.1 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1) ...
Setting up libportaudio2:amd64 (19.6.0-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...


In [9]:
from telegram.ext import Updater, CommandHandler, MessageHandler, RegexHandler
from telegram.ext import ConversationHandler, CallbackQueryHandler, Filters
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, InlineQueryResultVoice
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove
from encoder.params_model import model_embedding_size as speaker_embedding_size
from utils.argutils import print_args
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
import argparse
import torch
import sys
#from telegram_token import token
import logging

from torch.autograd import Variable
import torch

from array import array
from io import BytesIO
import numpy as np
import time
import os
import io
import subprocess

#from model_1 import transfer_style
#from model_2 import transform




logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO)

logger = logging.getLogger(__name__)

MENU, SET_STAT, ABOUT, MODEL, TTS, PREP = range(6)


def start(bot, update):
    """
    Start function. Calls when the /start command is called.
    """
    return menu(bot, update)


def menu(bot, update):
    """
    Main menu function.
    This will display the options from the main menu.
    """
    keyboard = [["Клонирование голоса", "О боте"]]
    reply_markup = ReplyKeyboardMarkup(keyboard,
                                       one_time_keyboard=True,
                                       resize_keyboard=True)
    user = update.message.from_user
    logger.info("Menu command shows for {}.".format(user.first_name))
    update.message.reply_text("Привет! Я бот, который может скопировать твою речь! \n"\
                                "Выбирай одну из функций ниже", reply_markup=reply_markup)
    return SET_STAT


def set_state(bot, update):
    """
    Set option selected from menu.
    """
    user = update.message.from_user
    if update.message.text == "Клонирование голоса":
        return redirect_to_prep(bot, update)
    elif update.message.text == "О боте":
        return about_bot(bot, update)
    else:
        return MENU

      
def redirect_to_prep(bot,update):
    update.message.reply_text("Отправь голосовое сообщение, произнеси: 'Six big devils from Japan quickly forgot how to waltz'")
    return PREP

def prep_func(bot, update):
    """
    Start
    """
    update.message.reply_text("Дай мне несколько секунд, не уходи")
    chat_id = update.message.chat_id
    print("Got voice message from {}".format(chat_id))
    

    file = bot.getFile(update.message.voice.file_id)
    file.download('voice.ogg')
    
    process = subprocess.run(['ffmpeg', '-i', 'voice.ogg', 'for-model.wav'])
    if process.returncode != 0:
      raise Exception("Something went wrong")

    device_id = torch.cuda.current_device()

    ## Load the models one by one.
    print("Preparing the encoder, the synthesizer and the vocoder...")
    encoder.load_model('encoder/saved_models/pretrained.pt')
    synthesizer = Synthesizer("synthesizer/saved_models/logs-pretrained/taco_pretrained")
    vocoder.load_model('vocoder/saved_models/pretrained/pretrained.pt')
    
    
    print("\tTesting the encoder...")
    encoder.embed_utterance(np.zeros(encoder.sampling_rate))
    
    # Create a dummy embedding. You would normally use the embedding that encoder.embed_utterance
    # returns, but here we're going to make one ourselves just for the sake of showing that it's
    # possible.
    embed = np.random.rand(speaker_embedding_size)
    # Embeddings are L2-normalized (this isn't important here, but if you want to make your own 
    # embeddings it will be).
    embed /= np.linalg.norm(embed)
    # The synthesizer can handle multiple inputs with batching. Let's create another embedding to 
    # illustrate that
    embeds = [embed, np.zeros(speaker_embedding_size)]
    texts = ["test 1", "test 2"]
    print("\tTesting the synthesizer... (loading the model will output a lot of text)")
    mels = synthesizer.synthesize_spectrograms(texts, embeds)
    
    # The vocoder synthesizes one waveform at a time, but it's more efficient for long ones. We 
    # can concatenate the mel spectrograms to a single one.
    mel = np.concatenate(mels, axis=1)
    # The vocoder can take a callback function to display the generation. More on that later. For 
    # now we'll simply hide it like this:
    #no_action = lambda *args: None

    
    num_generated = 0
    try:
        # Get the reference audio filepath
        message = "Reference voice: enter an audio filepath of a voice to be cloned (mp3, " \
                  "wav, m4a, flac, ...):\n"
        in_fpath = 'for-model.wav'
        
        
        ## Computing the embedding
        # First, we load the wav using the function that the speaker encoder provides. This is 
        # important: there is preprocessing that must be applied.
        
        # The following two methods are equivalent:
        # - Directly load from the filepath:
        preprocessed_wav = encoder.preprocess_wav(in_fpath)
        # - If the wav is already loaded:
        original_wav, sampling_rate = librosa.load(in_fpath)
        preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
        print("Loaded file succesfully")
        
        # Then we derive the embedding. There are many functions and parameters that the 
        # speaker encoder interfaces. These are mostly for in-depth research. You will typically
        # only use this function (with its default parameters):
        embed = encoder.embed_utterance(preprocessed_wav)
        print("Created the embedding")
        
        
        ## Generating the spectrogram
        text = 'perfect day for good thing'
        
        # The synthesizer works in batch, so you need to put your data in a list or numpy array
        texts = [text]
        embeds = [embed]
        # If you know what the attention layer alignments are, you can retrieve them here by
        # passing return_alignments=True
        specs = synthesizer.synthesize_spectrograms(texts, embeds)
        spec = specs[0]
        print("Created the mel spectrogram")
        
        
        ## Generating the waveform
        print("Synthesizing the waveform:")
        # Synthesizing the waveform is fairly straightforward. Remember that the longer the
        # spectrogram, the more time-efficient the vocoder.
        generated_wav = vocoder.infer_waveform(spec)
        
        
        ## Post-generation
        # There's a bug with sounddevice that makes the audio cut one second earlier, so we
        # pad it.
        generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
        
        # Play the audio (non-blocking)
        # sd.stop()
        # sd.play(generated_wav, synthesizer.sample_rate)
            
        # Save it on the disk
        print(generated_wav.dtype)
        librosa.output.write_wav('perfect_day_for_good_thing.wav', generated_wav.astype(np.float32), 
                                 synthesizer.sample_rate)
        
        
    except Exception as e:
        print("Caught exception: %s" % repr(e))
        print("Restarting\n")

    os.remove('for-model.wav')
    os.remove('voice.ogg')

    bot.send_audio(chat_id=chat_id, audio=open('perfect_day_for_good_thing.wav', 'rb'))
  
def synthesize(bot, update):
    bot.send_audio(chat_id=chat_id, audio=open('tests/test.mp3', 'rb'))
    return MENU

def about_bot(bot, update):
    """
    About bot. Displays info about Style Transfer Bot and his creator.
    """
    user = update.message.from_user
    logger.info("About bot info requested by {}.".format(user.first_name))
    bot.send_message(chat_id=update.message.chat_id, text=
    """
    Сайт бота: ###
	  Github разработчика: ###
	  Сайт разработчика: ###
	
    Этот бот создан в рамках проектной работы в Deep learning school.
    """)
    bot.send_message(chat_id=update.message.chat_id, text="Ты можешь вернуться обратно в меню с помощью команды /menu.")
    return MENU


def help(bot, update):
    """
    Help function.
    This displays a set of commands available for the bot.
    """
    user = update.message.from_user
    logger.info("User {} asked for help.".format(user.first_name))
    update.message.reply_text(
        "Используй команду /cancel , чтобы выйти из чата. \nИспользуй /start , чтобы перезагрузить бота.",
        reply_markup=ReplyKeyboardRemove())


def cancel(bot, update):
    """
    User cancelation function.
    Cancel conversation by user.
    """
    user = update.message.from_user
    logger.info("User {} canceled the conversation.".format(user.first_name))
    update.message.reply_text("Пока! Надеемся пообщаться с тобой ещё!",
                              reply_markup=ReplyKeyboardRemove())

    return ConversationHandler.END


def error(bot, update, error):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, error)


def main():
    """
    Main function.
    This function handles the conversation flow by setting
    states on each step of the flow. Each state has its own
    handler for the interaction with the user.
    """

    # Create the EventHandler and pass it your bot's token.
    updater = Updater(token, request_kwargs={'proxy_url': 'socks4://54.37.169.182:47625'})

    # Get the dispatcher to register handlers:
    dp = updater.dispatcher

    # Add conversation handler with predefined states:
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            MENU: [CommandHandler('menu', menu)],
            
            PREP: [MessageHandler(Filters.voice, prep_func)],
            
            TTS: [MessageHandler(Filters.text, synthesize)],

            SET_STAT: [RegexHandler(
                '^({}|{})$'.format(
                "Клонирование голоса", "О боте"),
                set_state)]
        },

        fallbacks=[CommandHandler('cancel', cancel),
                   CommandHandler('menu', menu),
                   CommandHandler('help', help)],
                   
        conversation_timeout = 900.0
    )

    dp.add_handler(conv_handler)

    # Log all errors:
    dp.add_error_handler(error)

    # Start Style transfer bot:
    updater.start_polling()

    # Run the bot until the user presses Ctrl-C or the process
    # receives SIGINT, SIGTERM or SIGABRT:
    updater.idle()


if __name__ == '__main__':
    print('ready')
    main()

ready
Got voice message from 279363416
Preparing the encoder, the synthesizer and the vocoder...
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt
Testing your configuration with small inputs.
	Testing the encoder...
	Testing the synthesizer... (loading the model will output a lot of text)
Constructing model: Tacotron


W0821 11:05:49.251325 140296016959232 ag_logging.py:145] Entity <bound method ZoneoutLSTMCell.__call__ of <synthesizer.models.modules.ZoneoutLSTMCell object at 0x7f991c8c65c0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method ZoneoutLSTMCell.__call__ of <synthesizer.models.modules.ZoneoutLSTMCell object at 0x7f991c8c65c0>>: ValueError: Failed to parse source code of <bound method ZoneoutLSTMCell.__call__ of <synthesizer.models.modules.ZoneoutLSTMCell object at 0x7f991c8c65c0>>, which Python reported as:
    def __call__(self, inputs, state, scope=None):
        """Runs vanilla LSTM Cell and applies zoneout.
        """
        # Apply vanilla LSTM
        output, new_state = self._cell(inputs, state, scope)

        if self.state_is_tuple:
            (prev_c, prev_h) = state
            (new_

    def __call__(self, inputs, state, scope=None):
        """Runs vanilla LSTM Cell and applies zoneout.
        """
        # Apply vanilla LSTM
        output, new_state = self._cell(inputs, state, scope)

        if self.state_is_tuple:
            (prev_c, prev_h) = state
            (new_c, new_h) = new_state
        else:
            num_proj = self._cell._num_units if self._cell._num_proj is None else \
				self._cell._num_proj
            prev_c = tf.slice(state, [0, 0], [-1, self._cell._num_units])
            prev_h = tf.slice(state, [0, self._cell._num_units], [-1, num_proj])
            new_c = tf.slice(new_state, [0, 0], [-1, self._cell._num_units])
            new_h = tf.slice(new_state, [0, self._cell._num_units], [-1, num_proj])

        # Apply zoneout
        if self.is_training:
            # nn.dropout takes keep_prob (probability to keep activations) not drop_prob (
			# probability to mask activations)!
            c = (1 - self._zoneout_cell) * tf.nn.dropout(new